In [1]:
!pip install nfl_data_py
!pip install cfbd

In [2]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import nfl_data_py as nfl
from collections import Counter
import requests
from bs4 import BeautifulSoup
import re
import cfbd
from scipy import stats
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

In [3]:
# cfb
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '5Hs/5kCfcbyDhcVBb98O+KQdxKfJsdG1QbYUlwGYp+ncYyNptbvHzsSz/r71nvCN'
configuration.api_key_prefix['Authorization'] = 'Bearer'
playersAPI = cfbd.PlayersApi(cfbd.ApiClient(configuration))
teamsAPI = cfbd.TeamsApi(cfbd.ApiClient(configuration))
team = teamsAPI.get_teams()
teams = pd.DataFrame([Team.to_dict() for Team in team])
teams = teams[teams['abbreviation'].notna()]

In [4]:
#data
draft = nfl.import_draft_picks()
draft = draft[draft['season'] >= 2000]
comb = nfl.import_combine_data()
winners = ["Trent Dilfer", "Tom Brady", "Brad Johnson", "Tom Brady", "Tom Brady", "Ben Roethlisberger", "Peyton Manning", "Eli Manning", "Ben Roethlisberger", "Drew Brees", "Aaron Rodgers", "Eli Manning", "Joe Flacco", "Russell Wilson", "Tom Brady", "Peyton Manning", "Tom Brady", "Nick Foles", "Tom Brady", "Patrick Mahomes", "Tom Brady", "Matthew Stafford"]
losers = ["Kerry Collins", "Kurt Warner", "Rich Gannon", "Jake Delhomme", "Donovan McNabb", "Matt Hasselbeck", "Rex Grossman", "Tom Brady", "Kurt Warner", "Peyton Manning", "Ben Roethlisberger", "Tom Brady", "Colin Kaepernick", "Peyton Manning", "Russell Wilson", "Cam Newton", "Matt Ryan", "Tom Brady", "Jared Goff", "Jimmy Garoppolo", "Patrick Mahomes", "Joe Burrow"]
years = np.arange(2001, 2023)

In [5]:
#Normalizing teams
draft.loc[draft['team'] == 'STL', 'team'] = 'LAR'
draft.loc[draft['team'] == 'LA', 'team'] = 'LAR'
draft.loc[draft['team'] == 'OAK', 'team'] = 'LV'
draft.loc[draft['team'] == 'SDG', 'team'] = 'LAC'

In [6]:
#COMBINE DATA
com = [comb]

url = requests.get('https://www.pro-football-reference.com/draft/2000-combine.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
combine2000 = pd.read_html(parse[start:end+8])[0]
combine2000['season'] = 2000
combine2000['10yd'] = np.nan
combine2000['20yd'] = np.nan
combine2000 = combine2000[['Player', 'School', 'Ht', 'Wt', 'Vertical', 'Broad Jump', 'Bench', 'Shuttle', '3Cone', '10yd', '20yd', '40yd', 'season', 'Pos']]
combine2000 = combine2000.rename(columns={'Player': 'name', 'School': 'college', 'Ht': 'height', 'Wt': 'weight', 'Vertical': 'vert', 'Broad Jump': 'broad', 'Bench': 'bench', 'Shuttle': 'shuttle', '3Cone': 'cone', 'Pos': 'position'})
for index, row in combine2000.iterrows():
  lst = [int(s) for s in re.findall(r'\b\d+\b', str(row['height']))]
  if len(lst) == 2:
    combine2000.loc[index, 'height']  = float((lst[0] * 12) + lst[1])
com.append(combine2000)

url = requests.get('https://www.pro-football-reference.com/draft/2001-combine.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
combine2001 = pd.read_html(parse[start:end+8])[0]
combine2001['season'] = 2001
combine2001['10yd'] = np.nan
combine2001['20yd'] = np.nan
combine2001 = combine2001[['Player', 'School', 'Ht', 'Wt', 'Vertical', 'Broad Jump', 'Bench', 'Shuttle', '3Cone', '10yd', '20yd', '40yd', 'season', 'Pos']]
combine2001 = combine2001.rename(columns={'Player': 'name', 'School': 'college', 'Ht': 'height', 'Wt': 'weight', 'Vertical': 'vert', 'Broad Jump': 'broad', 'Bench': 'bench', 'Shuttle': 'shuttle', '3Cone': 'cone', 'Pos': 'position'})
for index, row in combine2001.iterrows():
  lst = [int(s) for s in re.findall(r'\b\d+\b', str(row['height']))]
  if len(lst) == 2:
    combine2001.loc[index, 'height']  = float((lst[0] * 12) + lst[1])
com.append(combine2001)

url = requests.get('https://www.pro-football-reference.com/draft/2002-combine.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
combine2002 = pd.read_html(parse[start:end+8])[0]
combine2002['season'] = 2002
combine2002['10yd'] = np.nan
combine2002['20yd'] = np.nan
combine2002 = combine2002[['Player', 'School', 'Ht', 'Wt', 'Vertical', 'Broad Jump', 'Bench', 'Shuttle', '3Cone', '10yd', '20yd', '40yd', 'season', 'Pos']]
combine2002 = combine2002.rename(columns={'Player': 'name', 'School': 'college', 'Ht': 'height', 'Wt': 'weight', 'Vertical': 'vert', 'Broad Jump': 'broad', 'Bench': 'bench', 'Shuttle': 'shuttle', '3Cone': 'cone', 'Pos': 'position'})
for index, row in combine2002.iterrows():
  lst = [int(s) for s in re.findall(r'\b\d+\b', str(row['height']))]
  if len(lst) == 2:
    combine2002.loc[index, 'height']  = float((lst[0] * 12) + lst[1])
com.append(combine2002)

url = requests.get('https://www.pro-football-reference.com/draft/2021-combine.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
combine2021 = pd.read_html(parse[start:end+8])[0]
combine2021['season'] = 2021
combine2021['10yd'] = np.nan
combine2021['20yd'] = np.nan
combine2021 = combine2021[['Player', 'School', 'Ht', 'Wt', 'Vertical', 'Broad Jump', 'Bench', 'Shuttle', '3Cone', '10yd', '20yd', '40yd', 'season', 'Pos']]
combine2021 = combine2021.rename(columns={'Player': 'name', 'School': 'college', 'Ht': 'height', 'Wt': 'weight', 'Vertical': 'vert', 'Broad Jump': 'broad', 'Bench': 'bench', 'Shuttle': 'shuttle', '3Cone': 'cone', 'Pos': 'position'})
for index, row in combine2021.iterrows():
  lst = [int(s) for s in re.findall(r'\b\d+\b', str(row['height']))]
  if len(lst) == 2:
    combine2021.loc[index, 'height']  = float((lst[0] * 12) + lst[1])
com.append(combine2021)

url = requests.get('https://www.pro-football-reference.com/draft/2022-combine.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
combine2022 = pd.read_html(parse[start:end+8])[0]
combine2022['season'] = 2022
combine2022['10yd'] = np.nan
combine2022['20yd'] = np.nan
combine2022 = combine2022[['Player', 'School', 'Ht', 'Wt', 'Vertical', 'Broad Jump', 'Bench', 'Shuttle', '3Cone', '10yd', '20yd', '40yd', 'season', 'Pos']]
combine2022 = combine2022.rename(columns={'Player': 'name', 'School': 'college', 'Ht': 'height', 'Wt': 'weight', 'Vertical': 'vert', 'Broad Jump': 'broad', 'Bench': 'bench', 'Shuttle': 'shuttle', '3Cone': 'cone', 'Pos': 'position'})
for index, row in combine2022.iterrows():
  lst = [int(s) for s in re.findall(r'\b\d+\b', str(row['height']))]
  if len(lst) == 2:
    combine2022.loc[index, 'height']  = float((lst[0] * 12) + lst[1])
com.append(combine2022)

combine = pd.concat(com)
combine.drop(columns = ['position', 'college'], inplace = True)
draft = draft.rename(columns={'pfr_player_name': 'pfr_name'})
df = draft.merge(combine.reset_index(drop=True), how = 'left', left_on = ['pfr_name', 'season'], right_on = ['name', 'season'])

ParserError: Document is empty

In [ ]:
#CFB RANKS
colfb = []

url = requests.get('https://www.sports-reference.com/cfb/years/1999-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb1999 = pd.read_html(parse[start:end+8])[0]
cfb1999['Conference', 'cfbSeason'] = 1999
cfb1999['Conference', 'draftYear'] = 2000
colfb.append(cfb1999)

url = requests.get('https://www.sports-reference.com/cfb/years/2000-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2000 = pd.read_html(parse[start:end+8])[0]
cfb2000['Conference', 'cfbSeason'] = 2000
cfb2000['Conference', 'draftYear'] = 2001
colfb.append(cfb2000)

url = requests.get('https://www.sports-reference.com/cfb/years/2001-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2001 = pd.read_html(parse[start:end+8])[0]
cfb2001['Conference', 'cfbSeason'] = 2001
cfb2001['Conference', 'draftYear'] = 2002
colfb.append(cfb2001)

url = requests.get('https://www.sports-reference.com/cfb/years/2002-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2002 = pd.read_html(parse[start:end+8])[0]
cfb2002['Conference', 'cfbSeason'] = 2002
cfb2002['Conference', 'draftYear'] = 2003
colfb.append(cfb2002)

url = requests.get('https://www.sports-reference.com/cfb/years/2003-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2003 = pd.read_html(parse[start:end+8])[0]
cfb2003['Conference', 'cfbSeason'] = 2003
cfb2003['Conference', 'draftYear'] = 2004
colfb.append(cfb2003)

url = requests.get('https://www.sports-reference.com/cfb/years/2004-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2004 = pd.read_html(parse[start:end+8])[0]
cfb2004['Conference', 'cfbSeason'] = 2004
cfb2004['Conference', 'draftYear'] = 2005
colfb.append(cfb2004)

url = requests.get('https://www.sports-reference.com/cfb/years/2005-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2005 = pd.read_html(parse[start:end+8])[0]
cfb2005['Conference', 'cfbSeason'] = 2005
cfb2005['Conference', 'draftYear'] = 2006
colfb.append(cfb2005)

url = requests.get('https://www.sports-reference.com/cfb/years/2006-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2006 = pd.read_html(parse[start:end+8])[0]
cfb2006['Conference', 'cfbSeason'] = 2006
cfb2006['Conference', 'draftYear'] = 2007
colfb.append(cfb2006)

url = requests.get('https://www.sports-reference.com/cfb/years/2007-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2007 = pd.read_html(parse[start:end+8])[0]
cfb2007['Conference', 'cfbSeason'] = 2007
cfb2007['Conference', 'draftYear'] = 2008
colfb.append(cfb2007)

url = requests.get('https://www.sports-reference.com/cfb/years/2008-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2008 = pd.read_html(parse[start:end+8])[0]
cfb2008['Conference', 'cfbSeason'] = 2008
cfb2008['Conference', 'draftYear'] = 2009
colfb.append(cfb2008)

url = requests.get('https://www.sports-reference.com/cfb/years/2009-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2009 = pd.read_html(parse[start:end+8])[0]
cfb2009['Conference', 'cfbSeason'] = 2009
cfb2009['Conference', 'draftYear'] = 2010
colfb.append(cfb2009)

url = requests.get('https://www.sports-reference.com/cfb/years/2010-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2010 = pd.read_html(parse[start:end+8])[0]
cfb2010['Conference', 'cfbSeason'] = 2010
cfb2010['Conference', 'draftYear'] = 2011
colfb.append(cfb2010)

url = requests.get('https://www.sports-reference.com/cfb/years/2011-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2011 = pd.read_html(parse[start:end+8])[0]
cfb2011['Conference', 'cfbSeason'] = 2011
cfb2011['Conference', 'draftYear'] = 2012
colfb.append(cfb2011)

url = requests.get('https://www.sports-reference.com/cfb/years/2012-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2012 = pd.read_html(parse[start:end+8])[0]
cfb2012['Conference', 'cfbSeason'] = 2012
cfb2012['Conference', 'draftYear'] = 2013
colfb.append(cfb2012)

url = requests.get('https://www.sports-reference.com/cfb/years/2013-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2013 = pd.read_html(parse[start:end+8])[0]
cfb2013['Conference', 'cfbSeason'] = 2013
cfb2013['Conference', 'draftYear'] = 2014
colfb.append(cfb2013)

url = requests.get('https://www.sports-reference.com/cfb/years/2014-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2014 = pd.read_html(parse[start:end+8])[0]
cfb2014['Conference', 'cfbSeason'] = 2014
cfb2014['Conference', 'draftYear'] = 2015
colfb.append(cfb2014)

url = requests.get('https://www.sports-reference.com/cfb/years/2015-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2015 = pd.read_html(parse[start:end+8])[0]
cfb2015['Conference', 'cfbSeason'] = 2015
cfb2015['Conference', 'draftYear'] = 2016
colfb.append(cfb2015)

url = requests.get('https://www.sports-reference.com/cfb/years/2016-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2016 = pd.read_html(parse[start:end+8])[0]
cfb2016['Conference', 'cfbSeason'] = 2016
cfb2016['Conference', 'draftYear'] = 2017
colfb.append(cfb2016)

url = requests.get('https://www.sports-reference.com/cfb/years/2017-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2017 = pd.read_html(parse[start:end+8])[0]
cfb2017['Conference', 'cfbSeason'] = 2017
cfb2017['Conference', 'draftYear'] = 2018
colfb.append(cfb2017)

url = requests.get('https://www.sports-reference.com/cfb/years/2018-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2018 = pd.read_html(parse[start:end+8])[0]
cfb2018['Conference', 'cfbSeason'] = 2018
cfb2018['Conference', 'draftYear'] = 2019
colfb.append(cfb2018)

url = requests.get('https://www.sports-reference.com/cfb/years/2019-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2019 = pd.read_html(parse[start:end+8])[0]
cfb2019['Conference', 'cfbSeason'] = 2019
cfb2019['Conference', 'draftYear'] = 2020
colfb.append(cfb2019)

url = requests.get('https://www.sports-reference.com/cfb/years/2020-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2020 = pd.read_html(parse[start:end+8])[0]
cfb2020['Conference', 'cfbSeason'] = 2020
cfb2020['Conference', 'draftYear'] = 2021
colfb.append(cfb2020)

url = requests.get('https://www.sports-reference.com/cfb/years/2021-standings.html#standings')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
cfb2021 = pd.read_html(parse[start:end+8])[0]
cfb2021['Conference', 'cfbSeason'] = 2021
cfb2021['Conference', 'draftYear'] = 2022
colfb.append(cfb2021)


In [ ]:
#COMBINING RANKS
cfb = pd.concat(colfb)
cfb = cfb.droplevel(level = 0, axis = 1)
cfb.drop(['AP Post'], axis = 1, inplace = True)
cfb.columns = ['confL', 'confPCT', 'confW', 'cfbSeason', 'draftYear', 'overL', 'overPCT', 'overW', 'defPPG', 'offPPG', 'high', 'pre', 'final', 'SOS', 'SRS', 'rank', 'notes', 'college', 'conf']
cfb.drop(['notes'], axis = 1, inplace = True)
cfb["overW"] = cfb["overW"].astype(str)
cfb["college"] = cfb["college"].astype(str)
cfb = cfb[(cfb["overW"] != "Overall") | (cfb["college"] != "School")]
cfb = cfb[(cfb["overW"] != "Overall")]

In [ ]:
#College Data
coll = []

url = requests.get('https://www.pro-football-reference.com/years/2000/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2000 = pd.read_html(parse[start:end+8])[0]
college2000.insert(0, 'season', 2000)
coll.append(college2000)

url = requests.get('https://www.pro-football-reference.com/years/2001/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2001 = pd.read_html(parse[start:end+8])[0]
college2001.insert(0, 'season', 2001)
coll.append(college2001)

url = requests.get('https://www.pro-football-reference.com/years/2002/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2002 = pd.read_html(parse[start:end+8])[0]
college2002.insert(0, 'season', 2002)
coll.append(college2002)

url = requests.get('https://www.pro-football-reference.com/years/2003/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2003 = pd.read_html(parse[start:end+8])[0]
college2003.insert(0, 'season', 2003)
coll.append(college2003)

url = requests.get('https://www.pro-football-reference.com/years/2004/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2004 = pd.read_html(parse[start:end+8])[0]
college2004.insert(0, 'season', 2004)
coll.append(college2004)

url = requests.get('https://www.pro-football-reference.com/years/2005/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2005 = pd.read_html(parse[start:end+8])[0]
college2005.insert(0, 'season', 2005)
coll.append(college2005)

url = requests.get('https://www.pro-football-reference.com/years/2006/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2006 = pd.read_html(parse[start:end+8])[0]
college2006.insert(0, 'season', 2006)
coll.append(college2006)

url = requests.get('https://www.pro-football-reference.com/years/2007/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2007 = pd.read_html(parse[start:end+8])[0]
college2007.insert(0, 'season', 2007)
coll.append(college2007)

url = requests.get('https://www.pro-football-reference.com/years/2008/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2008 = pd.read_html(parse[start:end+8])[0]
college2008.insert(0, 'season', 2008)
coll.append(college2008)

url = requests.get('https://www.pro-football-reference.com/years/2009/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2009 = pd.read_html(parse[start:end+8])[0]
college2009.insert(0, 'season', 2009)
coll.append(college2009)

url = requests.get('https://www.pro-football-reference.com/years/2010/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2010 = pd.read_html(parse[start:end+8])[0]
college2010.insert(0, 'season', 2010)
coll.append(college2010)

url = requests.get('https://www.pro-football-reference.com/years/2011/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2011 = pd.read_html(parse[start:end+8])[0]
college2011.insert(0, 'season', 2011)
coll.append(college2011)

url = requests.get('https://www.pro-football-reference.com/years/2012/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2012 = pd.read_html(parse[start:end+8])[0]
college2012.insert(0, 'season', 2012)
coll.append(college2012)

url = requests.get('https://www.pro-football-reference.com/years/2013/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2013 = pd.read_html(parse[start:end+8])[0]
college2013.insert(0, 'season', 2013)
coll.append(college2013)

url = requests.get('https://www.pro-football-reference.com/years/2014/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2014 = pd.read_html(parse[start:end+8])[0]
college2014.insert(0, 'season', 2014)
coll.append(college2014)

url = requests.get('https://www.pro-football-reference.com/years/2015/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2015 = pd.read_html(parse[start:end+8])[0]
college2015.insert(0, 'season', 2015)
coll.append(college2015)

url = requests.get('https://www.pro-football-reference.com/years/2016/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2016 = pd.read_html(parse[start:end+8])[0]
college2016.insert(0, 'season', 2016)
coll.append(college2016)

url = requests.get('https://www.pro-football-reference.com/years/2017/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2017 = pd.read_html(parse[start:end+8])[0]
college2017.insert(0, 'season', 2017)
coll.append(college2017)

url = requests.get('https://www.pro-football-reference.com/years/2018/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2018 = pd.read_html(parse[start:end+8])[0]
college2018.insert(0, 'season', 2018)
coll.append(college2018)

url = requests.get('https://www.pro-football-reference.com/years/2019/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2019 = pd.read_html(parse[start:end+8])[0]
college2019.insert(0, 'season', 2019)
coll.append(college2019)

url = requests.get('https://www.pro-football-reference.com/years/2020/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2020 = pd.read_html(parse[start:end+8])[0]
college2020.insert(0, 'season', 2020)
coll.append(college2020)

url = requests.get('https://www.pro-football-reference.com/years/2021/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2021 = pd.read_html(parse[start:end+8])[0]
college2021.insert(0, 'season', 2021)
coll.append(college2021)

url = requests.get('https://www.pro-football-reference.com/years/2022/draft.htm')
parse = BeautifulSoup(url.text, "lxml").prettify()
start = parse.find("<table")
end = parse.find("</table")
college2022 = pd.read_html(parse[start:end+8])[0]
college2022.insert(0, 'season', 2022)
coll.append(college2022)

In [ ]:
#COMBINING COLLEGES
college = pd.concat(coll)
college = college.droplevel(level = 0, axis = 1)
college.columns = ['season', 'round',	'pick',	'team',	'pfr_name',	'pos',	'draft_age',	'retirement_year',	'all_pro_selections',	'pro_bowl_selections',	'years_starting',	'weighted_career_approx_value',	'drafted_team_approx_value',	'G',	'Cmp', 'Att', 'Yds', 'TD', 'Int', 'rAtt', 'rYds', 'rTD', 'xRec', 'xYds', 'xTD', 'Solo', 'yInt', 'Sk',	'college',	'link']
college.drop(['pfr_name', 'team', 'pos', 'Cmp', 'Att', 'Yds', 'TD', 'Int', 'rAtt', 'rYds', 'rTD', 'xRec', 'xYds', 'xTD', 'Solo', 'yInt', 'Sk', 'link'], axis = 1, inplace = True)
df.drop(['college'], axis = 1, inplace = True)
college = college[(college["round"] != "Rnd")]
college['round'] = college['round'].astype(int)
college['pick'] = college['pick'].astype(int)
df = df.merge(college.reset_index(drop=True), how = 'left', on = ['season', 'round', 'pick'])
df.loc[df['college'] == 'Ala-Birmingham', 'college'] = 'UAB'
df.loc[df['college'] == 'Alabama St.', 'college'] = 'Alabama State'
df.loc[df['college'] == 'Appalachian St.', 'college'] = 'Appalachian State'
df.loc[df['college'] == 'Arizona St.', 'college'] = 'Arizona State'
df.loc[df['college'] == 'Arkansas St.', 'college'] = 'Arkansas State'
df.loc[df['college'] == 'Ball St.', 'college'] = 'Ball State'
df.loc[df['college'] == 'Boise St.', 'college'] = 'Boise State'
df.loc[df['college'] == 'Boston Col.', 'college'] = 'Boston College'
df.loc[df['college'] == 'Bowling Green', 'college'] = 'Bowling Green State'
df.loc[df['college'] == 'Central Florida', 'college'] = 'UCF'
df.loc[df['college'] == 'Colorado St.', 'college'] = 'Colorado State'
df.loc[df['college'] == 'Delaware St.', 'college'] = 'Delaware State'
df.loc[df['college'] == 'East. Michigan', 'college'] = 'Eastern Michigan'
df.loc[df['college'] == 'East. Kentucky', 'college'] = 'Eastern Kentucky'
df.loc[df['college'] == 'East. Washington', 'college'] = 'Eastern Washington'
df.loc[df['college'] == 'Ferris St.', 'college'] = 'Ferris State'
df.loc[df['college'] == 'Florida St.', 'college'] = 'Florida State'
df.loc[df['college'] == 'Fresno St.', 'college'] = 'Fresno State'
df.loc[df['college'] == 'Georgia St.', 'college'] = 'Georgia State'
df.loc[df['college'] == 'Grambling St.', 'college'] = 'Grambling State'
df.loc[df['college'] == 'Idaho St.', 'college'] = 'Idaho State'
df.loc[df['college'] == 'Illinois St.', 'college'] = 'Illinois State'
df.loc[df['college'] == 'Iowa St.', 'college'] = 'Iowa State'
df.loc[df['college'] == 'Jackson St.', 'college'] = 'Jackson State'
df.loc[df['college'] == 'Jacksonville St.', 'college'] = 'Jacksonville State'
df.loc[df['college'] == 'Kansas St.', 'college'] = 'Kansas State'
df.loc[df['college'] == 'Kent St.', 'college'] = 'Kent State'
df.loc[df['college'] == 'Kentucky St.', 'college'] = 'Kentucky State'
df.loc[df['college'] == 'LA-Monroe', 'college'] = 'Louisiana-Monroe'
df.loc[df['college'] == 'McNeese St.', 'college'] = 'McNeese State'
df.loc[df['college'] == 'Michigan St.', 'college'] = 'Michigan State'
df.loc[df['college'] == 'Middle Tenn. St.', 'college'] = 'Middle Tennessee State'
df.loc[df['college'] == 'Mississippi St.', 'college'] = 'Mississippi State'
df.loc[df['college'] == 'Montana St.', 'college'] = 'Montana State'
df.loc[df['college'] == 'Morgan St.', 'college'] = 'Morgan State'
df.loc[df['college'] == 'Murray St.', 'college'] = 'Murray State'
df.loc[df['college'] == 'UNLV', 'college'] = 'Nevada-Las Vegas'
df.loc[df['college'] == 'New Mexico St.', 'college'] = 'New Mexico State'
df.loc[df['college'] == 'Nicholls St.', 'college'] = 'Nicholls State'
df.loc[df['college'] == 'Norfolk St.', 'college'] = 'Norfolk State'
df.loc[df['college'] == 'North Carolina St.', 'college'] = 'North Carolina State'
df.loc[df['college'] == 'North Dakota St.', 'college'] = 'North Dakota State'
df.loc[df['college'] == 'Ohio St.', 'college'] = 'Ohio State'
df.loc[df['college'] == 'Oklahoma St.', 'college'] = 'Oklahoma State'
df.loc[df['college'] == 'Mississippi', 'college'] = 'Ole Miss'
df.loc[df['college'] == 'Oregon St.', 'college'] = 'Oregon State'
df.loc[df['college'] == 'Penn St.', 'college'] = 'Penn State'
df.loc[df['college'] == 'Pittsburgh', 'college'] = 'Pitt'
df.loc[df['college'] == 'Sam Houston St.', 'college'] = 'Sam Houston State'
df.loc[df['college'] == 'San Diego St.', 'college'] = 'San Diego State'
df.loc[df['college'] == 'San Jose St.', 'college'] = 'San Jose State'
df.loc[df['college'] == 'South Carolina St.', 'college'] = 'South Carolina State'
df.loc[df['college'] == 'South Dakota St.', 'college'] = 'South Dakota State'
df.loc[df['college'] == 'Southern Miss', 'college'] = 'Southern Mississippi'
df.loc[df['college'] == 'Tennessee St.', 'college'] = 'Tennessee State'
df.loc[df['college'] == 'Texas-El Paso', 'college'] = 'UTEP'
df.loc[df['college'] == 'TCU', 'college'] = 'Texas Christian'
df.loc[df['college'] == 'Texas-San Antonio', 'college'] = 'UTSA'
df.loc[df['college'] == 'Texas St.', 'college'] = 'Texas State'
df.loc[df['college'] == 'Utah St.', 'college'] = 'Utah State'
df.loc[df['college'] == 'Virginia St.', 'college'] = 'Virginia State'
df.loc[df['college'] == 'Washington St.', 'college'] = 'Washington State'
df.loc[df['college'] == 'Weber St.', 'college'] = 'Weber State'
df.loc[df['college'] == 'Youngstown St.', 'college'] = 'Youngstown State'

In [ ]:
#MERGING RANKS TO DATAFRAMES
df = df.merge(cfb, how = 'left', left_on = ['season', 'college'], right_on = ['draftYear', 'college'])
df.drop_duplicates(keep='first', inplace=True, ignore_index=True)

#Correcting NaN, dropping extra column
df.loc[df['overL'] == 'Conference', 'overL'] = np.nan
df.loc[df['overPCT'] == 'Conference', 'overPCT'] = np.nan
df.loc[df['overW'] == 'Conference', 'overW'] = np.nan
df.loc[df['confL'] == 'Overall', 'confL'] = np.nan
df.loc[df['confW'] == 'Overall', 'confW'] = np.nan
df.loc[df['confPCT'] == 'Overall', 'confPCT'] = np.nan
df.loc[df['defPPG'] == 'Points Per Game', 'defPPG'] = np.nan
df.loc[df['offPPG'] == 'Points Per Game', 'offPPG'] = np.nan
df.loc[df['pre'] == 'Polls', 'pre'] = np.nan
df.loc[df['high'] == 'Polls', 'high'] = np.nan
df.loc[df['final'] == 'Polls', 'final'] = np.nan
df.loc[df['SOS'] == 'SRS', 'SOS'] = np.nan
df.loc[df['SRS'] == 'SRS', 'SRS'] = np.nan
df.drop(columns = ['name', 'draftYear', 'rank'], inplace = True)


In [ ]:
#Adding missing data
df['college'].fillna("Other", inplace = True)
df['conf'].fillna("Other", inplace = True)
temp = df.pop('college')
df.insert(7, 'college', temp)

missing_rushing = pd.DataFrame(columns = ['Name', 'Year', 'School', 'Conf', 'Class', 'Pos', 'G', 'Att', 'Yds', 'Avg', 'TD'])
missing_rushing.loc[0] = ['Tom Brady', 1999, 'Michigan', 'Big Ten', np.nan, 'QB', 11, 34, -31, -0.9, 1]
missing_rushing.loc[1] = ['Eli Manning', 2003, 'Mississippi', 'SEC', 'SR', 'QB', 13, 48, -28, -0.6, 3]
missing_rushing.loc[2] = ['Peyton Manning', 1997, 'Tennessee', 'SEC', np.nan, 'QB', 12, 49, -30, -0.6, 3]
missing_rushing.loc[3] = ['Drew Brees', 2000, 'Purdue', 'Big Ten', 'SR', 'QB', 12, 95, 521, 5.5, 5]
missing_rushing.loc[4] = ['Ben Roethlisberger', 2003, 'Miami (OH)', 'MAC', 'JR', 'QB', 14, 67, 111, 1.7, 3]

missing_passing = pd.DataFrame(columns = ['Name', 'Year', 'School', 'Conf', 'Class', 'Pos', 'G', 'Cmp', 'Att', 'Pct', 'Yds', 'YA', 'AYA', 'TD', 'Int', 'Rate'])
missing_passing.loc[0] = ['Tom Brady', 1999,	'Michigan', 'Big Ten', np.nan, 'QB', 11, 180, 295, 61.0, 2217, 7.5, 7.7,	16,	6, 138.0]
missing_passing.loc[1] = ['Eli Manning', 2003, 'Mississippi', 'SEC', 'SR', 'QB', 13, 275, 441, 62.4, 3600, 8.2, 8.5, 29, 10, 148.1]
missing_passing.loc[2] = ['Peyton Manning', 1997, 'Tennessee', 'SEC', np.nan, 'QB', 12, 287, 477, 60.2, 3819, 8.0, 8.5, 36, 11, 147.7]
missing_passing.loc[3] = ['Drew Brees', 2000, 'Purdue', 'Big Ten', 'SR', 'QB', 12, 309, 512, 60.4, 3668, 7.2, 7.1, 26, 12, 132.6]
missing_passing.loc[4] = ['Ben Roethlisberger', 2003, 'Miami (OH)', 'MAC', 'JR', 'QB', 14, 342, 495, 69.1, 4486, 9.1, 9.6, 37, 10, 165.8]


In [ ]:
#Andrew Luck
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Andrew Luck']['season']) - 1, team = 'Stanford')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Andrew Luck') & (data['category'] == 'passing')].reset_index(drop = True)

luck = df[df['pfr_name'] == 'Andrew Luck']
games = int(luck['overL']) + int(luck['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://www.si.com/.image/ar_16:9%2Cc_fill%2Ccs_srgb%2Cq_auto:good%2Cw_1200/MTY4MTI3ODc2OTc2NjgyMjU3/andrew-luck-beard-shotpng.png')
fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.358, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#A80532', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Trevor Lawrence
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Trevor Lawrence']['season']) - 1, team = 'Clemson')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Trevor Lawrence') & (data['category'] == 'passing')].reset_index(drop = True)

lawrence = df[df['pfr_name'] == 'Trevor Lawrence']
games = int(lawrence['overL']) + int(lawrence['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://static.wixstatic.com/media/ebbe7e_ff0fa1810c014d59be44a34c451ff93d~mv2.png/v1/fit/w_931%2Ch_816%2Cal_c/file.png')
fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.358, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#522d80', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Jameis Winston
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Jameis Winston']['season']) - 1, team = 'Florida State')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Jameis Winston') & (data['category'] == 'passing')].reset_index(drop = True)

winston = df[df['pfr_name'] == 'Jameis Winston']
games = int(winston['overL']) + int(winston['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://static.clubs.nfl.com/image/private/t_person_squared_mobile/f_png/saints/xiefsqpumhab0gxpav5a.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.365, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#ceb888', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Pat Mahomes
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Patrick Mahomes']['season']) - 1, team = 'Texas Tech')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Patrick Mahomes') & (data['category'] == 'passing')].reset_index(drop = True)

mahomes = df[df['pfr_name'] == 'Patrick Mahomes']
games = int(mahomes['overL']) + int(mahomes['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://www.swisswatchexpo.com/TheWatchClub/wp-content/uploads/2022/03/Copy-of-BLOG-Featured-Image-2-2.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.365, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#231f20', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Josh Allen
player_season_stats = playersAPI.get_player_season_stats(year = int(df[((df['pfr_name'] == 'Josh Allen') & (df['college'] == 'Wyoming'))]['season']) - 1, team = 'Wyoming')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Josh Allen') & (data['category'] == 'passing')].reset_index(drop = True)

allen = df[(df['pfr_name'] == 'Josh Allen') & (df['college'] == 'Wyoming')]
games = int(allen['overL']) + int(allen['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://www.si.com/.image/ar_1:1%2Cc_fill%2Ccs_srgb%2Cq_auto:good%2Cw_1200/MTg1NTExODk0ODE0MDQxMTcw/josh-allen.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = '#FFC425',
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 31, x = 0.3, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#FFC425', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Lamar Jackson
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Lamar Jackson']['season']) - 1, team = 'Louisville')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Lamar Jackson') & (data['category'] == 'passing')].reset_index(drop = True)

jackson = df[df['pfr_name'] == 'Lamar Jackson']
games = int(jackson['overL']) + int(jackson['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://brobible.com/wp-content/uploads/2022/05/NFL-Fans-Loved-Seeing-Lamar-Jackson-Clap-Back-At-Chris-Simms-On-Twitter.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.365, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#ad000a', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Derek Carr
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Derek Carr']['season']) - 1, team = 'Fresno State')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Derek Carr') & (data['category'] == 'passing')].reset_index(drop = True)

carr = df[df['pfr_name'] == 'Derek Carr']
games = int(carr['overL']) + int(carr['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://media.distractify.com/brand-img/KMG7y3Wdt/0x0/screen-shot-2020-09-22-at-124611-pm-1600793195914.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = '#002E6D',
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.365, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#002E6D', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Jalen Hurts
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Jalen Hurts']['season']) - 1, team = 'Oklahoma')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Jalen Hurts') & (data['category'] == 'passing')].reset_index(drop = True)

hurts = df[df['pfr_name'] == 'Jalen Hurts']
games = int(hurts['overL']) + int(hurts['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://mn2s-content.s3.eu-west-2.amazonaws.com/wp-content/uploads/2022/01/17170308/Jalen-Hurts.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = '#828A8F',
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.365, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#828A8F', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Russell Wilson
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Russell Wilson']['season']) - 1, team = 'Wisconsin')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Russell Wilson') & (data['category'] == 'passing')].reset_index(drop = True)

wilson = df[df['pfr_name'] == 'Russell Wilson']
games = int(wilson['overL']) + int(wilson['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://static.clubs.nfl.com/image/private/t_editorial_landscape_mobile/f_png/broncos/nmt6chscdx0ud7hx8iam.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['alt_color'].astype(str),
        edgecolor = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 25, x = 0.325, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#A00001', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Aaron Rodgers
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Aaron Rodgers']['season']) - 1, team = 'California')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Aaron Rodgers') & (data['category'] == 'passing')].reset_index(drop = True)

rodgers = df[df['pfr_name'] == 'Aaron Rodgers']
games = int(rodgers['overL']) + int(rodgers['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://stylecaster.com/wp-content/uploads/2021/12/aaron-rogers-retirement.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = '#003262',
        edgecolor = '#FDB515',
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 29, x = 0.37, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#FDB515', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#Kenny Pickett
player_season_stats = playersAPI.get_player_season_stats(year = int(df[df['pfr_name'] == 'Kenny Pickett']['season']) - 1, team = 'Pittsburgh')
data = pd.DataFrame([PlayerSeasonStat.to_dict() for PlayerSeasonStat in player_season_stats])
data = data[(data['player'] == 'Kenny Pickett') & (data['category'] == 'passing')].reset_index(drop = True)

pickett = df[df['pfr_name'] == 'Kenny Pickett']
games = int(pickett['overL']) + int(pickett['overW'])
p = np.array([['TD', float(data[data['stat_type'] == 'TD']['stat'])],
              ['INT', float(data[data['stat_type'] == 'INT']['stat'])],
              ['PCT',  float((data[data['stat_type'] == 'PCT']['stat'] * 100))],
              ['YPA', float(data[data['stat_type'] == 'YPA']['stat'])],
              ['ATT PG', float((data[data['stat_type'] == 'ATT']['stat'] / games))], 
              ['CMP PG',  float((data[data['stat_type'] == 'COMPLETIONS']['stat'] / games))]])
plot = pd.DataFrame(p, columns = ['stat', 'num'])
plot['num'] = plot['num'].astype(float)
plot = plot.reset_index(drop = True)
f = urlopen('https://www.si.com/.image/ar_1:1%2Cc_fill%2Ccs_srgb%2Cq_auto:good%2Cw_1200/MTg5MTE2Njg2MDk4NTA3NTQ0/kenny-pickett-steelers-site.png')

fig, ax = plt.subplots()
a = ax.bar(plot['stat'], plot['num'],
        color = teams[teams['school'].astype(str) == data.iloc[0]['team']]['color'].astype(str),
        edgecolor = '#FFB81C',
        linewidth = 3)
plt.suptitle(data.iloc[0]['player'], fontsize = 30, x = 0.35, y = 1.04)
ax.set_title("Final Year College Passing Stats", fontsize = 12, x = 0.28, y = 1)
plt.xlabel("Stat")
plt.ylabel("Number")
plt.ylim(0, 75)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(round(height, 1)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = '#FFB81C', 
      fontsize = 12)
im = plt.imread(f)
newax = fig.add_axes([0.64, 0.53, 0.275, 0.5], anchor='NE', zorder=1)
newax.imshow(im)
newax.axis('off')
plt.show()

In [ ]:
#WRs
qbs = df[df['position'] == 'QB']
qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].fillna(0)
qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].astype(float)
wrs = df[df['position'] == 'WR']
use = pd.DataFrame(columns = df.columns)
for i, r in qbs.iterrows():
  d = df[((df['season'] == r['season']) | (df['season'] == (r['season'] - 1)) | (df['season'] == (r['season'] + 1)))]
  d = d[(d['college'] == r['college']) & (d['position'] == 'WR')]
  d['qb_val'] = r['weighted_career_approx_value']
  d['qb_name'] = r['pfr_name']
  d['qb_round'] = r['round']
  use = pd.concat([use, d])

use['qb_val'] = use['qb_val'].astype(int)
use['pick'] = use['pick'].astype(int)
use = use.reset_index()
use.sort_values(['pick'], inplace = True)
use = use.reset_index()

best = use[(use['qb_val'] > 40)]
best.drop(['level_0'], axis = 1, inplace = True)
best = best.reset_index()

plt.scatter(use['pick'], use['qb_val'], c = use['round'], cmap = 'winter')
coeff = np.polyfit(use['pick'], use['qb_val'], 2)
x = np.linspace(best['pick'].iat[0], best['pick'].iat[-1], num = 1000)
Polynomial = np.polyval((5*coeff), x)
plt.suptitle("Selection Used On College WR vs QB Value in the NFL", ha = 'center')
plt.title("Does having good WRs in college actually hurt top QBs in the NFL?", fontsize = 10)
plt.xlabel("College WR Pick Drafted")
plt.ylabel("QB NFL Career Value (PFR)")
plt.annotate("*Annotation indicates draft round of the QB", (40, 183))
outliers = use[(use['qb_val'] > 100)]
outliers.drop(['level_0'], axis = 1, inplace = True)
outliers = outliers.reset_index()
for i, r in outliers.iterrows():
  plt.annotate(str(int(r['qb_round'])), (r['pick'] + 3, r['qb_val'] - 10))
plt.colorbar(orientation = "horizontal")
plt.show()

x1 = best[best['round'] == 1]['qb_val']
x2 = best[best['round'] == 2]['qb_val']
x3 = best[best['round'] == 3]['qb_val']
x4 = best[best['round'] == 4]['qb_val']
x5 = best[best['round'] == 5]['qb_val']
x6 = best[best['round'] == 6]['qb_val']
x7 = best[best['round'] == 7]['qb_val']
kwargs = dict(alpha=0.6, bins=5)
plt.hist(x1, **kwargs, color='g', label='round 1')
plt.hist(x2, **kwargs, color='y', label='round 2')
plt.hist(x3, **kwargs, color='r', label='round 3')
plt.hist(x4, **kwargs, color='b', label='round 4')
plt.hist(x5, **kwargs, color='c', label='round 5')
plt.hist(x6, **kwargs, color='m', label='round 6')
plt.hist(x7, **kwargs, color='k', label='round 7')
plt.xlabel("QB Value in NFL")
plt.ylabel("Frequency")
plt.legend(title = "WR selected in:")
good = qbs[qbs['weighted_career_approx_value'] > 40]
unused = good.shape[0] - len(set(best['qb_name']))
plt.suptitle("Value of QB vs College WR Round Drafted")
title = "{} qualified QBs with no college WR drafted".format(unused)
plt.title(title, fontsize = 10)


In [ ]:
qbs_avg = qbs.groupby(['pick'])['weighted_career_approx_value'].mean()
qbs_avg = np.array(qbs_avg)
empty_draft = pd.DataFrame(columns = ['round', 'pick', 'weighted_career_approx_value', 'count', 'weight'])
for r in range(1, 8):
  for p in range(1, 33):
    empty_draft.loc[len(empty_draft.index)] = [r, ((((r-1) * 32) + p)), 0, 0, 0] 

for i, r, in qbs.iterrows():
  empty_draft.loc[empty_draft['pick'] == r['pick'], 'weighted_career_approx_value'] += r['weighted_career_approx_value']
  empty_draft.loc[empty_draft['pick'] == r['pick'], 'count'] += 1

for i, r, in qbs.iterrows():
  empty_draft.loc[empty_draft['pick'] == r['pick'], 'weight'] += (r['weighted_career_approx_value'] * (1 / qbs.size))#empty_draft['count'])

empty_draft['pick'] = empty_draft['pick'].astype(float)
empty_draft['pick'] = empty_draft['count'].astype(float)
empty_draft['pick'] = empty_draft['weighted_career_approx_value'].astype(float)
empty_draft['pick'] = empty_draft['round'].astype(float)
empty_draft['pick'] = empty_draft['weight'].astype(float)
empty_draft['pick'] = empty_draft['weight'].fillna(0.0)
empty_draft['pick'] = empty_draft['pick'].fillna(0.0)
empty_draft['pick'] = empty_draft['count'].fillna(0.0)
empty_draft['pick'] = empty_draft['weighted_career_approx_value'].fillna(0.0)
empty_draft['pick'] = empty_draft['round'].fillna(0.0)

def division(n, d):
    return n / d if d else np.nan

def weighted_division(n, d):
    return (n / (1)) if d else np.nan
  
empty_draft['avg']  = empty_draft.apply(lambda x: division(x['weighted_career_approx_value'], x['count']), axis=1)
empty_draft['weighted_avg']  = empty_draft.apply(lambda x: weighted_division(x['weight'], x['count']), axis=1)
empty_draft['avg']  = empty_draft['avg'].round(1)
empty_draft['weighted_avg']  = empty_draft['weighted_avg'].round(4)

r1 = empty_draft[empty_draft['round'] == 1]['avg'].values
r2 = empty_draft[empty_draft['round'] == 2]['avg'].values
r3 = empty_draft[empty_draft['round'] == 3]['avg'].values
r4 = empty_draft[empty_draft['round'] == 4]['avg'].values
r5 = empty_draft[empty_draft['round'] == 5]['avg'].values
r6 = empty_draft[empty_draft['round'] == 6]['avg'].values
r7 = empty_draft[empty_draft['round'] == 7]['avg'].values

a = np.vstack([r1, r2, r3, r4, r5, r6, r7])
a = a.astype(float)

fig, ax = plt.subplots(figsize=(122, 18))
for i in range(len(a)):
    for j in range(len(a[i])):
        text = ax.text(j, i, a[i, j],
                       ha="center", va="center", color="k", fontsize = 40)
        
ax.set_yticklabels(range(0, 8))
ax.set_xticklabels(range(-4,33, 5))
ax.tick_params(labelsize = 50)
ax.set_title("Average QB Success Per Pick Heatmap", fontsize = 70)
plt.ylabel("Round Selected", fontsize = 55)
plt.xlabel("Pick Selected", fontsize = 55)
cm = plt.cm.get_cmap('spring').reversed()
ax.imshow(a, cmap = cm)

In [ ]:
empty_draft.loc[empty_draft['count'] == 0.0, 'count'] = np.nan
c1 = empty_draft[empty_draft['round'] == 1]['count'].values
c2 = empty_draft[empty_draft['round'] == 2]['count'].values
c3 = empty_draft[empty_draft['round'] == 3]['count'].values
c4 = empty_draft[empty_draft['round'] == 4]['count'].values
c5 = empty_draft[empty_draft['round'] == 5]['count'].values
c6 = empty_draft[empty_draft['round'] == 6]['count'].values
c7 = empty_draft[empty_draft['round'] == 7]['count'].values

c = np.vstack([c1, c2, c3, c4, c5, c6, c7])
c = c.astype(float)

fig, ax = plt.subplots(figsize=(122, 18))
for i in range(len(c)):
    for j in range(len(c[i])):
        text = ax.text(j, i, c[i, j],
                       ha="center", va="center", color="k", fontsize = 40)
        
ax.set_yticklabels(range(0, 8))
ax.set_xticklabels(range(-4,33, 5))
ax.tick_params(labelsize = 50)
ax.set_title("Total QBs Selected Per Pick Heatmap", fontsize = 70)
plt.ylabel("Round Selected", fontsize = 55)
plt.xlabel("Pick Selected", fontsize = 55)
cm = plt.cm.get_cmap('spring').reversed()
ax.imshow(c, cmap = cm)

In [ ]:
empty_draft['pick'] = empty_draft['count'].fillna(0.0)
w1 = empty_draft[empty_draft['round'] == 1]['weighted_avg'].values
w2 = empty_draft[empty_draft['round'] == 2]['weighted_avg'].values
w3 = empty_draft[empty_draft['round'] == 3]['weighted_avg'].values
w4 = empty_draft[empty_draft['round'] == 4]['weighted_avg'].values
w5 = empty_draft[empty_draft['round'] == 5]['weighted_avg'].values
w6 = empty_draft[empty_draft['round'] == 6]['weighted_avg'].values
w7 = empty_draft[empty_draft['round'] == 7]['weighted_avg'].values

w = np.vstack([w1, w2, w3, w4, w5, w6, w7])
w = w.astype(float)

fig, ax = plt.subplots(figsize=(122, 18))
for i in range(len(w)):
    for j in range(len(w[i])):
        text = ax.text(j, i, w[i, j],
                       ha="center", va="center", color="k", fontsize = 30)
        
ax.set_yticklabels(range(0, 8))
ax.set_xticklabels(range(-4,33, 5))
ax.tick_params(labelsize = 50)
ax.set_title("Weighted Average QB Success Per Pick Heatmap", fontsize = 70)
plt.ylabel("Round Selected", fontsize = 55)
plt.xlabel("Pick Selected", fontsize = 55)
cm = plt.cm.get_cmap('hot').reversed()
ax.imshow(w, cmap = 'cool')

In [ ]:
win = pd.DataFrame({"pfr_name": np.flip(winners), "year": np.flip(years)})
win.drop_duplicates(subset = ['pfr_name'], keep = 'last', inplace = True, ignore_index = True)
win_graph = win.merge(df.reset_index(drop=True), how = 'left', on = ['pfr_name'])
win_graph.loc[win_graph['pfr_name'] == 'Brad Johnson', 'season'] = 1992
win_graph.loc[win_graph['pfr_name'] == 'Peyton Manning', 'season'] = 1998
win_graph.loc[win_graph['pfr_name'] == 'Trent Dilfer', 'season'] = 1994
win_graph['first_super_bowl'] = win_graph['year'] - win_graph['season']# - 1

fig, ax = plt.subplots()
a = ax.bar(win_graph['pfr_name'], win_graph['first_super_bowl'],
        color = '#E3CF57',
        edgecolor = 'k',
        linewidth = 3)
ax.set_title("Year of First Super Bowl Win by QB", fontsize = 20)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'k', 
      fontsize = 10)
plt.xlabel("Quarterback")
plt.ylabel("Years until first Super Bowl win")
plt.xticks(rotation = 45, ha = 'right')

In [ ]:
#Change vet QB season to first year on new team, rookie to first year primary starter, removed Nick Foles
win_graph['first_year'] = win_graph['season']
win_graph.loc[win_graph['pfr_name'] == 'Trent Dilfer', 'first_year'] = 2000
win_graph.loc[win_graph['pfr_name'] == 'Tom Brady', 'first_year'] = 2001
win_graph.loc[win_graph['pfr_name'] == 'Brad Johnson', 'first_year'] = 2001
win_graph.loc[win_graph['pfr_name'] == 'Eli Manning', 'first_year'] = 2005
win_graph.loc[win_graph['pfr_name'] == 'Drew Brees', 'first_year'] = 2006
win_graph.loc[win_graph['pfr_name'] == 'Aaron Rodgers', 'first_year'] = 2008
win_graph.loc[win_graph['pfr_name'] == 'Drew Brees', 'first_year'] = 2006
win_graph = win_graph[win_graph['pfr_name'] != 'Nick Foles']
win_graph.loc[win_graph['pfr_name'] == 'Patrick Mahomes', 'first_year'] = 2018
win_graph.loc[win_graph['pfr_name'] == 'Matthew Stafford', 'first_year'] = 2021
win_graph['first_super_bowl_adjusted'] = win_graph['year'] - win_graph['first_year']# - 1

fig, ax = plt.subplots()
a = ax.bar(win_graph['pfr_name'], win_graph['first_super_bowl_adjusted'],
        color = '#E3CF57',
        edgecolor = 'k',
        linewidth = 3)
ax.set_title("Year of First Super Bowl Win by QB (Adjusted)", fontsize = 16)
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'k', 
      fontsize = 10)
plt.xlabel("Quarterback")
plt.ylabel("Years until first Super Bowl win")
plt.xticks(rotation = 45, ha = 'right')

In [ ]:
#Divide 0
def inc(n, d):
    return (((n - d) / d) * 100) if d else 0.0

#Winner Graphs
win_graph.loc[win_graph['pfr_name'] == 'Eli Manning', 'team'] = 'NYG'
winners_first_team = win_graph[win_graph['first_super_bowl'] <= 5]

#drafted_before_winner
drafted_before_winner = [df[(r['season']  - 4 < df['season']) &
              (df['season'] <= r['season']) &
              (df['team'] == r['team'])] for i, r in winners_first_team.iterrows()]

drafted_before_winner = pd.concat(drafted_before_winner, ignore_index = True)
before_count = Counter(drafted_before_winner['category'])

#drafted_after_winner
drafted_after_winner = [df[(r['season'] < df['season']) &
              (df['season'] <= r['season'] + 4) &
              (df['team'] == r['team'])] for i, r in winners_first_team.iterrows()]

drafted_after_winner = pd.concat(drafted_after_winner, ignore_index = True)
after_count = Counter(drafted_after_winner['category'])

#Seperating major capital from minor capital
all_before123 = drafted_before_winner[drafted_before_winner['round'] <= 3]
before_count123 = Counter(all_before123['category'])

all_before4567 = drafted_before_winner[drafted_before_winner['round'] > 3]
before_count4567 = Counter(all_before4567['category'])

all_after123 = drafted_after_winner[drafted_after_winner['round'] <= 3]
after_count123 = Counter(all_after123['category'])

all_after4567 = drafted_after_winner[drafted_after_winner['round'] > 3]
after_count4567 = Counter(all_after4567['category'])

#Preparing for plotting
pos = ['QB', 'RB', 'WR', 'TE', 'OL', 'DL', 'LB', 'DB', 'K', 'P']
before_vals = [before_count[p] for p in pos]
after_vals = [after_count[p] for p in pos]
before_vals123 = [before_count123[p] for p in pos]
after_vals123 = [after_count123[p] for p in pos]
before_vals4567 = [before_count4567[p] for p in pos]
after_vals4567 = [after_count4567[p] for p in pos]

#Plotting
fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals)), before_vals,
        color = 'lightgreen',
        edgecolor = 'darkgreen',
        linewidth = 3)
ax.set_title("Draft Capital Spent Before a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(before_vals)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkgreen', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals)), after_vals,
        color = 'lightgreen',
        edgecolor = 'darkgreen',
        linewidth = 3)
ax.set_title("Draft Capital Spent After a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(after_vals)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkgreen', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals123)), before_vals123,
        color = 'paleturquoise',
        edgecolor = 'darkslategray',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent Before a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(before_vals123)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkslategray', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals123)), after_vals123,
        color = 'paleturquoise',
        edgecolor = 'darkslategray',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent After a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(after_vals123)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkslategray', weight = 'bold', 
      fontsize = 10)
plt.show()

increase = []
for i in range(len(after_vals123)):
  increase.append(inc(float(after_vals123[i]), float(before_vals123[i])))

fig, ax = plt.subplots()
a = ax.bar(range(len(increase)), increase,
        color = 'paleturquoise',
        edgecolor = 'darkslategray',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent After a Super Bowl QB (% Increase)", fontsize = 16)
plt.xticks(range(len(increase)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
plt.axhline(y = 0, color = 'k', linestyle = '-')
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkslategray', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals4567)), before_vals4567,
        color = 'thistle',
        edgecolor = 'indigo',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent Before a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(before_vals4567)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'indigo', weight = 'bold', 
      fontsize = 10)
plt.show()


fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals4567)), after_vals4567,
        color = 'thistle',
        edgecolor = 'indigo',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent After a Super Bowl QB", fontsize = 16)
plt.xticks(range(len(after_vals4567)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'indigo', weight = 'bold', 
      fontsize = 10)
plt.show()

increase = []
for i in range(len(after_vals4567)):
  increase.append(inc(float(after_vals4567[i]), float(before_vals4567[i])))

fig, ax = plt.subplots()
a = ax.bar(range(len(increase)), increase,
        color = 'thistle',
        edgecolor = 'indigo',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent After a Super Bowl QB (% Increase)", fontsize = 16)
plt.xticks(range(len(increase)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
plt.axhline(y = 0, color = 'k', linestyle = '-')
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'indigo', weight = 'bold', 
      fontsize = 10)
plt.show()


In [ ]:
#loser Graphs
losers_copy = df.copy()

losers_copy = losers_copy[losers_copy['position'] == 'QB']
losers_copy = losers_copy.reset_index()

for i, r in winners_first_team.iterrows():
  losers_copy = losers_copy[losers_copy['pfr_name'] != r['pfr_name']]

#drafted_before_loser
drafted_before_loser = [df[(r['season']  - 4 < df['season']) &
              (df['season'] <= r['season']) &
              (df['team'] == r['team'])] for i, r in losers_copy.iterrows()]

drafted_before_loser = pd.concat(drafted_before_loser, ignore_index = True)
before_count = Counter(drafted_before_loser['category'])

#drafted_after_loser
drafted_after_loser = [df[(r['season'] < df['season']) &
              (df['season'] <= r['season'] + 4) &
              (df['team'] == r['team'])] for i, r in losers_copy.iterrows()]

drafted_after_loser = pd.concat(drafted_after_loser, ignore_index = True)
after_count = Counter(drafted_after_loser['category'])

#Seperating major capital from minor capital
all_before123 = drafted_before_loser[drafted_before_loser['round'] <= 3]
before_count123 = Counter(all_before123['category'])

all_before4567 = drafted_before_loser[drafted_before_loser['round'] > 3]
before_count4567 = Counter(all_before4567['category'])

all_after123 = drafted_after_loser[drafted_after_loser['round'] <= 3]
after_count123 = Counter(all_after123['category'])

all_after4567 = drafted_after_loser[drafted_after_loser['round'] > 3]
after_count4567 = Counter(all_after4567['category'])

#Preparing for plotting
pos = ['QB', 'RB', 'WR', 'TE', 'OL', 'DL', 'LB', 'DB', 'K', 'P']
before_vals = [before_count[p] for p in pos]
after_vals = [after_count[p] for p in pos]
before_vals123 = [before_count123[p] for p in pos]
after_vals123 = [after_count123[p] for p in pos]
before_vals4567 = [before_count4567[p] for p in pos]
after_vals4567 = [after_count4567[p] for p in pos]

#Plotting
fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals)), before_vals,
        color = 'mistyrose',
        edgecolor = 'darkred',
        linewidth = 3)
ax.set_title("Draft Capital Spent Before a Non Super Bowl QB", fontsize = 10)
plt.xticks(range(len(before_vals)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkred', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals)), after_vals,
        color = 'mistyrose',
        edgecolor = 'darkred',
        linewidth = 3)
ax.set_title("Draft Capital Spent After a Non Super Bowl QB", fontsize = 10)
plt.xticks(range(len(after_vals)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkred', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals123)), before_vals123,
        color = 'peachpuff',
        edgecolor = 'orangered',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent Before a Non Super Bowl QB", fontsize = 16)
plt.xticks(range(len(before_vals123)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'orangered', weight = 'bold', 
      fontsize = 10)
plt.show()


fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals123)), after_vals123,
        color = 'peachpuff',
        edgecolor = 'orangered',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent After a Non Super Bowl QB", fontsize = 16)
plt.xticks(range(len(after_vals123)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'orangered', weight = 'bold', 
      fontsize = 10)
plt.show()

increase = []
for i in range(len(after_vals123)):
  increase.append(inc(float(after_vals123[i]), float(before_vals123[i])))

fig, ax = plt.subplots()
a = ax.bar(range(len(increase)), increase,
        color = 'peachpuff',
        edgecolor = 'orangered',
        linewidth = 3)
ax.set_title("Major Draft Capital Spent After a Non Super Bowl QB (% Increase)", fontsize = 16)
plt.xticks(range(len(increase)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
plt.axhline(y = 0, color = 'k', linestyle = '-')
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'orangered', weight = 'bold', 
      fontsize = 10)
plt.show()

fig, ax = plt.subplots()
a = ax.bar(range(len(before_vals4567)), before_vals4567,
        color = 'cornsilk',
        edgecolor = 'darkgoldenrod',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent Before a Non Super Bowl QB", fontsize = 16)
plt.xticks(range(len(before_vals4567)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkgoldenrod', weight = 'bold', 
      fontsize = 10)
plt.show()


fig, ax = plt.subplots()
a = ax.bar(range(len(after_vals4567)), after_vals4567,
        color = 'cornsilk',
        edgecolor = 'darkgoldenrod',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent After a Non Super Bowl QB", fontsize = 16)
plt.xticks(range(len(after_vals4567)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkgoldenrod', weight = 'bold', 
      fontsize = 10)
plt.show()

increase = []
for i in range(len(after_vals4567)):
  increase.append(inc(float(after_vals4567[i]), float(before_vals4567[i])))

fig, ax = plt.subplots()
a = ax.bar(range(len(increase)), increase,
        color = 'cornsilk',
        edgecolor = 'darkgoldenrod',
        linewidth = 3)
ax.set_title("Minor Draft Capital Spent After a Non Super Bowl QB (% Increase)", fontsize = 16)
plt.xticks(range(len(increase)), pos)
ax.set_ylabel("Number of players picked")
ax.set_xlabel("Position")
plt.axhline(y = 0, color = 'k', linestyle = '-')
for x in a:
  height = x.get_height()
  ax.annotate('{}'.format(int(height)),
      xy=(x.get_x() + x.get_width() / 2, height / 2),
      ha='center', color = 'darkgoldenrod', weight = 'bold', 
      fontsize = 10)
plt.show()

In [ ]:
qbs = df[df['position'] == 'QB']
qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].fillna(0)
qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].astype(int)
plt.scatter(qbs['pick'], qbs['weighted_career_approx_value'], c = qbs['round'], cmap = 'winter')
x = np.polyfit(qbs["pick"], qbs["weighted_career_approx_value"], 1)
p = np.poly1d(x)
plt.plot(qbs["pick"], p(qbs["pick"]),"r--")
plt.title("Weighted Career Approximate Value (PFR) vs Pick Drafted")
plt.xlabel("Pick Drafted")
plt.ylabel("Weighted Career Approximate Value (PFR)")
plt.show()


qbs['final'] = qbs['final'].fillna(26)
qbs['final'] = qbs['final'].astype(int)
qbs.sort_values('final', inplace = True)
plt.scatter(qbs['final'], qbs['weighted_career_approx_value'], c = qbs['round'], cmap = 'winter')
x = np.polyfit(qbs["final"], qbs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(qbs["final"], p(qbs["final"]),"r-")
plt.suptitle("Weighted Career Approximate Value (PFR) vs Final CFB Ranking")
plt.title("Unranked teams included at rank 26", fontsize = 10)
plt.xlabel("Final CFB Ranking")
plt.ylabel("Weighted Career Approximate Value (PFR)")
plt.colorbar()
plt.annotate("*Colormap: round QB selected", (11.5, 183))
plt.show()




In [ ]:
rbs = df[df['category'] == 'RB']
wrs = df[df['category'] == 'WR']
tes = df[df['category'] == 'TE']
ols = df[df['category'] == 'OL']
dls = df[df['category'] == 'DL']
lbs = df[df['category'] == 'LB']
dbs = df[df['category'] == 'DB']
ks = df[df['category'] == 'K']
ps = df[df['category'] == 'P']

qbs['final'] = qbs['final'].fillna(26)
qbs['final'] = qbs['final'].astype(int)
rbs['final'] = rbs['final'].fillna(26)
rbs['final'] = rbs['final'].astype(int)
wrs['final'] = wrs['final'].fillna(26)
wrs['final'] = wrs['final'].astype(int)
tes['final'] = tes['final'].fillna(26)
tes['final'] = tes['final'].astype(int)
ols['final'] = ols['final'].fillna(26)
ols['final'] = ols['final'].astype(int)
dls['final'] = dls['final'].fillna(26)
dls['final'] = dls['final'].astype(int)
lbs['final'] = lbs['final'].fillna(26)
lbs['final'] = lbs['final'].astype(int)
dbs['final'] = dbs['final'].fillna(26)
dbs['final'] = dbs['final'].astype(int)
ks['final'] = ks['final'].fillna(26)
ks['final'] = ks['final'].astype(int)
ps['final'] = ps['final'].fillna(26)
ps['final'] = ps['final'].astype(int)

qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].fillna(0)
qbs['weighted_career_approx_value'] = qbs['weighted_career_approx_value'].astype(int)
rbs['weighted_career_approx_value'] = rbs['weighted_career_approx_value'].fillna(0)
rbs['weighted_career_approx_value'] = rbs['weighted_career_approx_value'].astype(int)
wrs['weighted_career_approx_value'] = wrs['weighted_career_approx_value'].fillna(0)
wrs['weighted_career_approx_value'] = wrs['weighted_career_approx_value'].astype(int)
tes['weighted_career_approx_value'] = tes['weighted_career_approx_value'].fillna(0)
tes['weighted_career_approx_value'] = tes['weighted_career_approx_value'].astype(int)
ols['weighted_career_approx_value'] = ols['weighted_career_approx_value'].fillna(0)
ols['weighted_career_approx_value'] = ols['weighted_career_approx_value'].astype(int)
dls['weighted_career_approx_value'] = dls['weighted_career_approx_value'].fillna(0)
dls['weighted_career_approx_value'] = dls['weighted_career_approx_value'].astype(int)
lbs['weighted_career_approx_value'] = lbs['weighted_career_approx_value'].fillna(0)
lbs['weighted_career_approx_value'] = lbs['weighted_career_approx_value'].astype(int)
dbs['weighted_career_approx_value'] = dbs['weighted_career_approx_value'].fillna(0)
dbs['weighted_career_approx_value'] = dbs['weighted_career_approx_value'].astype(int)
ks['weighted_career_approx_value'] = ks['weighted_career_approx_value'].fillna(0)
ks['weighted_career_approx_value'] = ks['weighted_career_approx_value'].astype(int)
ps['weighted_career_approx_value'] = ps['weighted_career_approx_value'].fillna(0)
ps['weighted_career_approx_value'] = ps['weighted_career_approx_value'].astype(int)

qbs.sort_values('final', inplace = True)
rbs.sort_values('final', inplace = True)
wrs.sort_values('final', inplace = True)
tes.sort_values('final', inplace = True)
ols.sort_values('final', inplace = True)
dls.sort_values('final', inplace = True)
lbs.sort_values('final', inplace = True)
dbs.sort_values('final', inplace = True)
ks.sort_values('final', inplace = True)
ps.sort_values('final', inplace = True)

cmap = plt.get_cmap('plasma')
colors = cmap(np.linspace(0, 1, 10))

x = np.polyfit(qbs["final"], qbs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(qbs["final"], p(qbs["final"]), color = colors[0], label = "QB", marker = '+')

x = np.polyfit(rbs["final"], rbs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(rbs["final"], p(rbs["final"]), color = colors[1], label = "RB")

x = np.polyfit(wrs["final"], wrs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(wrs["final"], p(wrs["final"]), color = colors[2], label = "WR")

x = np.polyfit(tes["final"], tes["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(tes["final"], p(tes["final"]), color = colors[3], label = "TE")

x = np.polyfit(ols["final"], ols["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(ols["final"], p(ols["final"]), color = colors[4], label = "OL")

x = np.polyfit(dls["final"], dls["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(dls["final"], p(dls["final"]), color = colors[5], label = "DL")

x = np.polyfit(lbs["final"], lbs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(lbs["final"], p(lbs["final"]), color = colors[6], label = "LB")

x = np.polyfit(dbs["final"], dbs["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(dbs["final"], p(dbs["final"]), color = colors[7], label = "DB")

x = np.polyfit(ks["final"], ks["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(ks["final"], p(ks["final"]), color = colors[8], label = "K")

x = np.polyfit(ps["final"], ps["weighted_career_approx_value"], 4)
p = np.poly1d(x)
plt.plot(ps["final"], p(ps["final"]), color = colors[9], label = "P")

plt.legend(loc = 1, bbox_to_anchor=(1.2, 1.02), title = 'Position')

plt.suptitle("Weighted Career Approximate Value (PFR) vs Final CFB Ranking")
plt.title("Unranked teams included at rank 26", fontsize = 10)
plt.xlabel("Final CFB Ranking")
plt.ylabel("Weighted Career Approximate Value (PFR)")
plt.show()